In [1]:
import sys; sys.path.append('..')
import random, math, os
import pyzx as zx
from pyzx.circuit import FSim, XPhase, ZPhase
from fractions import Fraction
import numpy as np
np.set_printoptions(linewidth=500)
%config InlineBackend.figure_format = 'svg'

In [2]:
c = zx.Circuit.load("test2.qsim")
lim = 256
fslim = 6
focus = 43
i = 0
for g in c.gates:
    if isinstance(g, FSim):
        if g.control == focus: focus = g.target
        elif g.target == focus: focus = g.control
    elif isinstance(g, XPhase) and g.target == focus:
        g.phase = Fraction(0,1)
        i += 1
i

28

In [3]:
len(c.gates)

4717

In [6]:
# Some gate stats
{
 'total':len(c.gates),
 'fsim':sum(1 for g in c.gates if type(g) is zx.gates.FSim),
 't':sum(1 for g in c.gates if type(g) is zx.gates.ZPhase and g.phase.denominator==4),
 't-like':sum(1 for g in c.gates if type(g) is zx.gates.ZPhase and g.phase.denominator>2)
}

{'total': 712, 'fsim': 60, 't': 114, 't-like': 354}

In [6]:
qubit_map = [
    (0,0), (1, 0), (2, 0), (3, 0),
    (0,1), (1, 1), (2, 1), (3, 1),
    (0,2), (1, 2), (2, 2), (3, 2),
    (0,3), (1, 3), (2, 3), (3, 3)
]
def mp(q,r):
    q0 = min(max(0,math.floor(q)), len(qubit_map)-1)
    offset = q - q0
    coord = qubit_map[q0]
    return (3*(coord[0]+offset)+(0.6 * coord[1]), 3*r+(0.6 * coord[1]))
mp(8, 2)

(1.2, 7.2)

In [24]:
g = c.to_graph(zh=True)
# for v in g.vertices():
#     (q,r) = mp(g.qubit(v),g.row(v))
#     g.set_qubit(v, q)
#     g.set_row(v, r)
#g.apply_state(g.qubit_count() * "+")
#g.apply_effect(g.qubit_count() * "+")

In [25]:
zx.d3.draw(g)

In [26]:
g1 = g.copy()

In [27]:
zx.full_reduce(g1)
#g1.normalise()

In [28]:
zx.d3.draw(g1, labels=True)

In [14]:
zx.hsimplify.hpivot_simp(g1)

hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  1 iterations
par_hbox: 2.  1 iterations
hpivot: 1.  

In [15]:
#g1.normalise()
zx.d3.draw(g1)

In [ ]:
def real_tcount(g):
    return sum(1 for v in g.vertices() if g.type(v) == 1 and g.phase(v).denominator == 4)
def deg3(g):
    return sum(1 for v in g.vertices() if g.vertex_degree(v) > 2)
def num_nodes(g, typ):
    return sum(1 for v in g.vertices() if g.type(v) == typ)
def hbox_crossing(g, bnd):
    n = 0
    for v in g.vertices():
        if g.type(v) != 3: continue
        ws = g.neighbours(v)
        if any(g.qubit(w) <= bnd for w in ws) and any(g.qubit(w) > bnd for w in ws):
            n += 1
    return n
def edge_crossing(g, bnd):
    n = 0
    for e in g.edges():
        s,t = g.edge_st(e)
        if g.type(s) == 3 or g.type(t) == 3: continue
        if ((g.qubit(s) <= bnd and g.qubit(t) > bnd) or
            (g.qubit(s) > bnd and g.qubit(t) <= bnd)):
            n += 1
    return n

In [ ]:
(g1.num_vertices(), deg3(g1))

In [ ]:
b = 10
[(hbox_crossing(g1, b), edge_crossing(g1, b))
 for b in range(1, 52)]

In [10]:
f = open('demi.graphml', 'w')
f.write(zx.to_graphml(g1))
f.close()

In [ ]:
hd = [v for v in g.vertices() if g.vertex_degree(v) > 2]
(sum(g.vertex_degree(v) for v in hd)/float(len(hd)), len(hd))

In [ ]:
hd = [v for v in g1.vertices() if g1.vertex_degree(v) > 2]
(sum(g1.vertex_degree(v) for v in hd)/float(len(hd)), len(hd))